Install necessary libraries

In [1]:
!pip -q install langchain openai tiktoken pypdf faiss-cpu langchainhub
!pip install -U langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
ERROR: pip's dependency resol

Declare OPENAI API KEY

In [2]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda

In [5]:
PATH = "/content/1.pdf" #your PDF path

loader = PyPDFLoader(PATH)
pages = loader.load_and_split()

#pages will contain metadata like page and source pdf as well

Now split the full text into chunks

In [6]:
# Splitting up the text into smaller chunks for indexing
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200, #striding over the text
    length_function = len,
)


In [7]:
docs_new = text_splitter.split_documents(pages)


In [8]:
len(docs_new)

49

In [9]:
docs_new[17].page_content

'ChengduEbyteElectronicTechnologyCo,;Ltd E32-433T30Dusermanual\nCopyright©2012–2019，ChengduEbyteElectronicTechnologyCo.,Ltd. 8\n5Functiondescription\n5.1Fixedtransmission\n5.2Broadcastingtransmission\n5.3Broadcastingaddress\n\uf06cForexample:SettheaddressofmoduleAas0xFFFF,andthechannelas0x04;\n\uf06cWhenmoduleisthetransmitter(transparenttransmission),allmodulesunderchannel0x04willreceivethedata,the'

In [10]:
docs_new[17].metadata

{'source': '/content/1.pdf', 'page': 8}

Now Embeddings of the data

In [11]:
embeddings = OpenAIEmbeddings()

In [12]:
db = FAISS.from_documents(docs_new, embeddings)

Retriever

In [13]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [14]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7e3269b2b580>, search_kwargs={'k': 6})

In [15]:
query = "What is E32?"
retrieved_docs = retriever.invoke(query)
print((retrieved_docs))

[Document(page_content='E32-915T30DSX1276915M 30 80.3~19.2kDIP24*43SMA-K\nE32-170T30DSX1278170M 30 80.3k~9.6kDIP24*43SMA-K\nE32-868T20DSX1276868M 20 30.3~19.2kDIP21*36SMA-K\nE32-915T20DSX1276915M 20 30.3~19.2kDIP21*36SMA-K\nE32-433T20DCSX1278433M 20 30.3k~19.2kDIP21*36SMA-K\nE32-433T30DSX1278433M 30 80.3k~19.2kDIP24*43SMA-K', metadata={'source': '/content/1.pdf', 'page': 18}), Document(page_content='ChengduEbyteElectronicTechnologyCo,;Ltd E32-433T30Dusermanual\nCopyright©2012–2019，ChengduEbyteElectronicTechnologyCo.,Ltd. 3\nDisclaimer\nEBYTEreservesallrightstothisdocumentandtheinformationcontainedherein.\nProducts,names,logosanddesignsdescribedhereinmayinwholeorinpartbe\nsubjecttointellectualpropertyrights.Reproduction,use,modificationordisclosureto\nthirdpartiesofthisdocumentoranypartthereofwithouttheexpresspermissionof\nEBYTEisstrictlyprohibited.\nTheinformationcontainedhereinisprovided“asis”andEBYTEassumesno\nliabilityfortheuseoftheinformation.Nowarranty,eitherexpressorimplied,isgiv

Select prompt

In [16]:
prompt = hub.pull("rlm/rag-prompt")

Define LLM

In [ ]:
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
rag_chain_from_docs = (
    RunnablePassthrough.assign(context = (lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_from_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer = rag_chain_from_docs)

In [ ]:
rag_chain_from_source.invoke("what is E32?")

{'context': [Document(page_content='ChengduEbyteElectronicTechnologyCo,;Ltd E32-433T30Dusermanual\nCopyright©2012–2019，ChengduEbyteElectronicTechnologyCo.,Ltd. 3\nDisclaimer\nEBYTEreservesallrightstothisdocumentandtheinformationcontainedherein.\nProducts,names,logosanddesignsdescribedhereinmayinwholeorinpartbe\nsubjecttointellectualpropertyrights.Reproduction,use,modificationordisclosureto\nthirdpartiesofthisdocumentoranypartthereofwithouttheexpresspermissionof\nEBYTEisstrictlyprohibited.\nTheinformationcontainedhereinisprovided“asis”andEBYTEassumesno\nliabilityfortheuseoftheinformation.Nowarranty,eitherexpressorimplied,isgiven,\nincludingbutnotlimited,withrespecttotheaccuracy,correctness,reliabilityand\nfitnessforaparticularpurposeoftheinformation.Thisdocumentmayberevisedby\nEBYTEatanytime.Formostrecentdocuments,visitwww.ebyte.com.', metadata={'source': '/content/1.pdf', 'page': 3}),
  Document(page_content='E32-915T30DSX1276915M 30 80.3~19.2kDIP24*43SMA-K\nE32-170T30DSX1278170M 30 80

Extras


To understand RunnableParallel and RunnablePassthrough in LCEL

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

def add(a):
  return a+1

chain = RunnablePassthrough.assign(pass_3 = lambda x: x["check"])

run = RunnableParallel(
    {"check" : RunnablePassthrough()},
    passed = RunnablePassthrough(),

    pass_2 = RunnablePassthrough.assign(mul = lambda x: x["num"]*3),
    pass_3 = RunnablePassthrough.assign(mul_1 = lambda x: add(x["num"]))

).assign(ans = chain)

In [ ]:
run.invoke({"num" : 3})

{'check': {'num': 3},
 'passed': {'num': 3},
 'pass_2': {'num': 3, 'mul': 9},
 'pass_3': {'num': 3, 'mul_1': 4},
 'ans': {'check': {'num': 3},
  'passed': {'num': 3},
  'pass_2': {'num': 3, 'mul': 9},
  'pass_3': {'num': 3}}}

In [ ]:
rag_chain_from_docs = (
    RunnablePassthrough.assign(context_1 = (lambda x: format_docs(x["context"])))
    #| prompt
    #| llm
    #| StrOutputParser()
)

rag_chain_from_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer = rag_chain_from_docs)

In [ ]:
rag_chain_from_source.invoke("what is E32?")

{'context': [Document(page_content='ChengduEbyteElectronicTechnologyCo,;Ltd E32-433T30Dusermanual\nCopyright©2012–2019，ChengduEbyteElectronicTechnologyCo.,Ltd. 3\nDisclaimer\nEBYTEreservesallrightstothisdocumentandtheinformationcontainedherein.\nProducts,names,logosanddesignsdescribedhereinmayinwholeorinpartbe\nsubjecttointellectualpropertyrights.Reproduction,use,modificationordisclosureto\nthirdpartiesofthisdocumentoranypartthereofwithouttheexpresspermissionof\nEBYTEisstrictlyprohibited.\nTheinformationcontainedhereinisprovided“asis”andEBYTEassumesno\nliabilityfortheuseoftheinformation.Nowarranty,eitherexpressorimplied,isgiven,\nincludingbutnotlimited,withrespecttotheaccuracy,correctness,reliabilityand\nfitnessforaparticularpurposeoftheinformation.Thisdocumentmayberevisedby\nEBYTEatanytime.Formostrecentdocuments,visitwww.ebyte.com.', metadata={'source': '/content/1.pdf', 'page': 3}),
  Document(page_content='E32-915T30DSX1276915M 30 80.3~19.2kDIP24*43SMA-K\nE32-170T30DSX1278170M 30 80